In [1]:
from tqdm.auto import tqdm
import numpy as np

import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from gymnasium.utils.play import play

# env = gym.make("ALE/BankHeist-v5", render_mode="human", frameskip=2)
# # play(env, zoom=3)
# env.reset()
# env.render()

env = gym.make("ALE/BankHeist-v5", render_mode="grayscale", frameskip=2)
env.reset()


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

# plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True

In [2]:
def convert_observation(observation: np.ndarray) -> np.ndarray[np.uint8]:
    converted_observation = observation[41:177, 12:148]


    bank_boxes = np.array([[[0, 0, 0, 0],
                            [0, 0, 142, 142],
                            [0, 142, 142, 142],
                            [0, 142, 0, 142]],
                           [[0, 0, 0, 0],
                            [142, 142, 0, 0],
                            [142, 142, 142, 0],
                            [142, 0, 142, 0]],
                           [[0, 142, 0, 142],
                            [0, 142, 0, 142],
                            [0, 142, 0, 142],
                            [142, 142, 142, 142]],
                           [[142, 0, 142, 0],
                            [142, 0, 142, 0],
                            [142, 0, 142, 0],
                            [142, 142, 142, 142]]], dtype=np.uint8)

    # print(bank_boxes)

    shape = (converted_observation.shape[0] // 2, converted_observation.shape[1] // 2)

    new_observation = np.zeros(shape, dtype=np.uint8)

    for num_row in range(0, shape[0]):
        num_row_start = num_row * 4
        num_row_finish = num_row_start + 4

        for num_col in range(0, shape[1]):
            num_col_start = num_col * 4
            num_col_finish = num_col_start + 4

            box = converted_observation[num_row_start:num_row_finish, num_col_start:num_col_finish]

            new_obs_row_start = num_row_start // 2
            new_obs_row_finish = num_row_finish // 2
            new_obs_col_start = num_col_start // 2
            new_obs_col_finish = num_col_finish // 2

            if np.all(box != 0): # стены
                new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 1
                # print(box[0,0])
                continue

            elif np.all(box == 0): # пол
                new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 0
                continue

            elif np.any(box == 110): # персонаж
                new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 2
                continue

            elif np.any(box == 41): # полиция
                new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 3
                continue

            elif np.any(box in bank_boxes): # банки и цифры реварда
                is_bank = False
                for bank_box in bank_boxes:
                    if np.all(box == bank_box):
                        is_bank = True
                        new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 4
                        break
                if is_bank:
                    continue
                else:
                    new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 3

            elif np.any(box != 0): # бимба
                new_observation[new_obs_row_start:new_obs_row_finish, new_obs_col_start:new_obs_col_finish] = 5
                continue
            # else:
            # print((num_row_start, num_row_finish), (num_col_start, num_col_finish), box, sep='\n')

    return new_observation

In [3]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__() # [b, 1, 136, 136]
        self.max_pool = nn.MaxPool2d(kernel_size=8, stride=2) # [b, 1, 68, 68]
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2) # [b, 16, 68, 68]
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # [b, 32, 136, 136]
        self.layer1 = nn.Linear(1 * 68 * 68, 2048) 
        self.layer2 = nn.Linear(2048, 512)
        self.layer3 = nn.Linear(512, n_actions)

    def forward(self, x):
        x = x.unsqueeze(1)
        # x = self.conv1(x)
        # x = self.max_pool(x)
        # x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return F.softmax(self.layer3(x), 1)

In [5]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.2
EPS_END = 0.2
EPS_DECAY = 10000
TAU = 0.005
LR = 1e-4

# Number of actions from gym action space
n_actions = 6
# Get the number of state observations
state, info = env.reset()
n_observations = len(state.reshape(-1, 1))

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(100000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
                    math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            temp = policy_net(state).max(1).indices[0]
            if temp == 1:
                return torch.tensor([[0]], device=device, dtype=torch.long)
            else:
                return torch.tensor([[temp]], device=device, dtype=torch.long)
            
    else:
        temp = np.random.randint(5)
        if temp == 1:
            return torch.tensor([[0]], device=device, dtype=torch.long)
        else:
            return torch.tensor([[temp]], device=device, dtype=torch.long)


episode_durations = []
reward_count = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                       if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    with torch.cuda.amp.autocast(dtype=torch.half):
        state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [7]:
if torch.cuda.is_available():
    num_episodes = 1000
else:
    num_episodes = 50

for i_episode in tqdm(range(num_episodes), desc="Training"):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = convert_observation(state)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    reward_sum = 0
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        observation = convert_observation(observation)
        if reward > 0:
            print(reward)
            reward_sum += reward
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 − τ)θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            print(f"Reward for episode {i_episode}: {reward_sum}")
            print(f"Episode {i_episode} duration: {t + 1}")
            reward_count.append(reward_sum)
            episode_durations.append(t + 1)
            # plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

10.0
10.0
Reward for episode 0: 20.0
Episode 0 duration: 8803
10.0
10.0
10.0
10.0
Reward for episode 1: 40.0
Episode 1 duration: 7256
10.0
Reward for episode 2: 10.0
Episode 2 duration: 9123
10.0
20.0
10.0
Reward for episode 3: 40.0
Episode 3 duration: 6646
10.0
10.0
20.0
Reward for episode 4: 40.0
Episode 4 duration: 7055
10.0
10.0
Reward for episode 5: 20.0
Episode 5 duration: 6679
10.0
10.0
10.0
10.0
Reward for episode 6: 40.0
Episode 6 duration: 7075
10.0
10.0
10.0
Reward for episode 7: 30.0
Episode 7 duration: 7011
10.0
Reward for episode 8: 10.0
Episode 8 duration: 6455
10.0
10.0
Reward for episode 9: 20.0
Episode 9 duration: 6722
Reward for episode 10: 0
Episode 10 duration: 8739
10.0
Reward for episode 11: 10.0
Episode 11 duration: 2387
10.0
10.0
10.0
Reward for episode 12: 30.0
Episode 12 duration: 5170
10.0
Reward for episode 13: 10.0
Episode 13 duration: 4707
10.0
20.0
10.0
10.0
Reward for episode 14: 50.0
Episode 14 duration: 4963
10.0
10.0
10.0
Reward for episode 15: 30.0


In [16]:
print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

Complete


In [8]:
torch.save(policy_net.state_dict(), "policy_net_2.pt")

In [10]:
def plot_rewards(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(reward_count, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [11]:
plot_rewards(show_result=True)

In [15]:
durations_t = torch.tensor(reward_count, dtype=torch.float)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.plot(durations_t.numpy())
if len(durations_t) >= 100:
    means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
    means = torch.cat((torch.zeros(99), means))
    plt.plot(means.numpy())
plt.show()